# Part 1. Compound data aquisition from ChEMBL database

## Introduction

In this module, we will explore how to retrieve compound data from the ChEMBL database for a specific molecular target. This data can be utilized for various chemoinformatics tasks such as similarity search, clustering, and machine learning.

## Background

ChEMBL is a meticulously curated database housing bioactive molecules exhibiting drug-like properties. Through its web resource client accessible via Python, we can access a plethora of compound activity measures including IC50 (half maximal inhibitory concentration), pIC50 (negative logarithm of the IC50, facilitating comparison of related values), EC50, etc. These measures provide crucial information for developing systems capable of predicting a molecule's likelihood to be a candidate drug.

## Objective

The primary objective of this module is to obtain compound data from ChEMBL for a specific molecular target  with Uniprod ID P42336 and filter available bioactivity data.

In [9]:
# import dependencies
import pandas as pd
import math
import rdkit
from tqdm.auto import tqdm
from chembl_webresource_client.new_client import new_client
from pandas import DataFrame
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, PandasTools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

In [10]:
#define local variables
HERE = Path(_dh[-1])
DATA = HERE / "data"

## Create the resource objects for API access

In [11]:
targets_api = new_client.target
compounds_api = new_client.molecule
bioactivities_api = new_client.activity

In [12]:
type(targets_api)  #show the type of the object

chembl_webresource_client.query_set.QuerySet

## Molecular target data

We chose the molecular target with UniProt ID P42336 because it represents the human Phosphatidylinositol 4,5-bisphosphate 3-kinase catalytic subunit alpha isoform, commonly referred to as PIK3CA. PIK3CA is an enzyme belonging to the phosphoinositide-3-kinase (PI3K) family, pivotal in regulating diverse cellular processes such as growth, survival, proliferation, and migration. Comprising 1068 amino acids and with a molecular weight of approximately 124 kDa, PIK3CA can interact with a regulatory subunit known as p85alpha, which modulates its activity and localization. Mutations in PIK3CA are prevalent in various cancers including breast, ovarian, colorectal, and brain cancers, often resulting in increased kinase activity and aberrant signaling, thereby contributing to tumorigenesis. Given its significance in cancer pathogenesis, PIK3CA serves as a promising target for cancer therapy.

In [13]:
uniprot_id = "P42336" 

## Acquire target data from ChEMBL database

In [14]:
# Get target information from ChEMBL but restrict it to specified class only
targets = targets_api.get(target_components__accession=uniprot_id).only(              ##variable that contains the results of the query
    "target_chembl_id", "organism", "pref_name", "target_type"
)
print(f'The type of the targets is "{type(targets)}"')

The type of the targets is "<class 'chembl_webresource_client.query_set.QuerySet'>"


In [15]:
# use pandas to convert data to a dataframe
targets = pd.DataFrame(targets)
targets.head()

,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,PI3-kinase p110-alpha subunit,CHEMBL4005,SINGLE PROTEIN
1,Homo sapiens,PI3-kinase p110-alpha/p85-alpha,CHEMBL2111367,PROTEIN COMPLEX
2,Homo sapiens,PI3-kinase class I,CHEMBL3559703,PROTEIN COMPLEX GROUP
3,Homo sapiens,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",CHEMBL3885616,PROTEIN FAMILY


## Select the target (ChEMBL ID)

In this context, considering our research focus on PIK3CA, the target should be "Phosphatidylinositol 4,5-bisphosphate 3-kinase catalytic subunit alpha" or "PI3-kinase p110-alpha subunit" (represented by target_chembl_id: CHEMBL4005). This specific protein isoform, encoded by the PIK3CA gene, is the primary catalytic subunit of PI3K and plays a central role in various cellular processes, including growth, survival, and proliferation. By selecting this single protein isoform as our target, we can effectively investigate its interactions with ligands and gain insights into potential therapeutic interventions targeting PIK3CA-mediated signaling pathways.

In [24]:
target = targets.iloc[0]
print(target)

organism                             Homo sapiens
pref_name           PI3-kinase p110-alpha subunit
target_chembl_id                       CHEMBL4005
target_type                        SINGLE PROTEIN
Name: 0, dtype: object


Let's save the target

In [25]:
target_id = target.target_chembl_id
print(f"The target ChEMBL ID is {target_id}")

The target ChEMBL ID is CHEMBL4005


## Retrieve Bioactivity Data for Tested Ligands

In this crucial step, we acquire bioactivity data relevant to the selected molecular target. By querying the database, we obtain information specifically tailored to human proteins, focusing on IC50 values, which represent the half maximal inhibitory concentration. We filter the data to include only exact measurements (denoted by '='), ensuring accuracy and reliability. Furthermore, we exclusively consider binding-based assays ('B'), which provide insights into the interaction between ligands and the target protein. This meticulous filtering process ensures that the acquired bioactivity data aligns precisely with our research objectives, facilitating robust analysis and interpretation.

In [26]:
bioactivities = bioactivities_api.filter(
    target_chembl_id=target_id, type="IC50", relation="=", assay_type="B"
).only(
    "activity_id",
    "assay_chembl_id",
    "assay_description",
    "assay_type",
    "molecule_chembl_id",
    "type",
    "standard_units",
    "relation",
    "standard_value",
    "target_chembl_id",
    "target_organism",
)

print(f"Length and type of bioactivities object: {len(bioactivities)}, {type(bioactivities)}")

Length and type of bioactivities object: 5422, <class 'chembl_webresource_client.query_set.QuerySet'>


In [27]:
# every element aquired holds some information
print(f"Length and type of first element: {len(bioactivities[0])}, {type(bioactivities[0])}")
bioactivities[0]

Length and type of first element: 13, <class 'dict'>


{'activity_id': 1410288,
 'assay_chembl_id': 'CHEMBL829496',
 'assay_description': 'Inhibition of Phosphatidylinositol 3-kinase',
 'assay_type': 'B',
 'molecule_chembl_id': 'CHEMBL435507',
 'relation': '=',
 'standard_units': 'nM',
 'standard_value': '2400.0',
 'target_chembl_id': 'CHEMBL4005',
 'target_organism': 'Homo sapiens',
 'type': 'IC50',
 'units': 'uM',
 'value': '2.4'}

## Download Bioactivity data from ChEMBL

In [28]:
# save obtained information into a pandas dataframe.
bioactivities_df = pd.DataFrame.from_records(bioactivities)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.head()

DataFrame shape: (5423, 13)


,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type,units,value
0,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50,uM,2.4
1,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50,uM,2.4
2,1412178,CHEMBL830973,Inhibition of human recombinant p110 alpha Pho...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50,uM,2.3
3,1412285,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL104468,=,nM,13000.0,CHEMBL4005,Homo sapiens,IC50,uM,13.0
4,1459741,CHEMBL831920,Inhibition of Phosphatidylinositol 3-kinase p1...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50,uM,2.3


In [31]:
bioactivities_df.drop(["units", "value"], axis=1, inplace=True)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type
0,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50
1,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50
2,1412178,CHEMBL830973,Inhibition of human recombinant p110 alpha Pho...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50
3,1412285,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL104468,=,nM,13000.0,CHEMBL4005,Homo sapiens,IC50
4,1459741,CHEMBL831920,Inhibition of Phosphatidylinositol 3-kinase p1...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50


## Preprocess and filter bioactivity data

In [33]:
bioactivities_df.dtypes

activity_id            int64
assay_chembl_id       object
assay_description     object
assay_type            object
molecule_chembl_id    object
relation              object
standard_units        object
standard_value        object
target_chembl_id      object
target_organism       object
type                  object
dtype: object

1. Convert datatype of “standard_value” from “object” to “float”

In [34]:
bioactivities_df = bioactivities_df.astype({"standard_value" : "float64"})

In [35]:
bioactivities_df.dtypes

activity_id             int64
assay_chembl_id        object
assay_description      object
assay_type             object
molecule_chembl_id     object
relation               object
standard_units         object
standard_value        float64
target_chembl_id       object
target_organism        object
type                   object
dtype: object

2. Check for missingness

In [38]:
bioactivities_df.isna().sum()

activity_id           0
assay_chembl_id       0
assay_description     0
assay_type            0
molecule_chembl_id    0
relation              0
standard_units        0
standard_value        0
target_chembl_id      0
target_organism       0
type                  0
dtype: int64

3. Delete duplicates

In [40]:
bioactivities_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")

DataFrame shape: (4790, 11)


In [41]:
bioactivities_df.reset_index(drop=True, inplace=True)
bioactivities_df.head()


,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type
0,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50
1,1412178,CHEMBL830973,Inhibition of human recombinant p110 alpha Pho...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50
2,1412285,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL104468,=,nM,13000.0,CHEMBL4005,Homo sapiens,IC50
3,1472257,CHEMBL831920,Inhibition of Phosphatidylinositol 3-kinase p1...,B,CHEMBL188678,=,nM,5000.0,CHEMBL4005,Homo sapiens,IC50
4,1734324,CHEMBL871583,Inhibition of human PI3Kalpha,B,CHEMBL379156,=,nM,940.0,CHEMBL4005,Homo sapiens,IC50


4. Rename Columns

In [42]:
bioactivities_df.rename(
    columns={"standard_value": "IC50", "standard_units": "units"}, inplace=True
)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,units,IC50,target_chembl_id,target_organism,type
0,1410288,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL435507,=,nM,2400.0,CHEMBL4005,Homo sapiens,IC50
1,1412178,CHEMBL830973,Inhibition of human recombinant p110 alpha Pho...,B,CHEMBL98350,=,nM,2300.0,CHEMBL4005,Homo sapiens,IC50
2,1412285,CHEMBL829496,Inhibition of Phosphatidylinositol 3-kinase,B,CHEMBL104468,=,nM,13000.0,CHEMBL4005,Homo sapiens,IC50
3,1472257,CHEMBL831920,Inhibition of Phosphatidylinositol 3-kinase p1...,B,CHEMBL188678,=,nM,5000.0,CHEMBL4005,Homo sapiens,IC50
4,1734324,CHEMBL871583,Inhibition of human PI3Kalpha,B,CHEMBL379156,=,nM,940.0,CHEMBL4005,Homo sapiens,IC50


## Fetch Compound Data (Molecular Structures) from ChEMBL

In this part, we proceed to acquire molecular structure data of compounds from the ChEMBL database. These compounds are of particular interest due to their interactions with our target protein, PIK3CA. By accessing the ChEMBL database, we retrieve comprehensive information regarding the chemical structures of these compounds, which is crucial for further analysis and interpretation. Molecular structure data, often represented in the form of canonical SMILES (Simplified Molecular Input Line Entry System), provides insights into the chemical composition and configuration of each compound. This data serves as the foundation for various computational and chemoinformatics analyses, including ligand-based drug design, virtual screening, and structure-activity relationship (SAR) studies. By fetching compound data from ChEMBL, we ensure access to a diverse and relevant set of compounds for our research on PIK3CA-targeted drug discovery and development.

In [43]:
compounds_provider = compounds_api.filter(
    molecule_chembl_id__in=list(bioactivities_df["molecule_chembl_id"])
).only("molecule_chembl_id", "molecule_structures")

In [44]:
compounds = list(tqdm(compounds_provider))

  0%|          | 0/4790 [00:00<?, ?it/s]

In [45]:
compounds_df = pd.DataFrame.from_records(
    compounds,
)
print(f"DataFrame shape: {compounds_df.shape}")

DataFrame shape: (4790, 2)


In [46]:
compounds_df.head()

,molecule_chembl_id,molecule_structures
0,CHEMBL428496,{'canonical_smiles': 'COC[C@H]1OC(=O)c2coc3c2[...
1,CHEMBL273832,{'canonical_smiles': 'COC[C@H]1OC(=O)c2coc3c2[...
2,CHEMBL535,{'canonical_smiles': 'CCN(CC)CCNC(=O)c1c(C)[nH...
3,CHEMBL113,{'canonical_smiles': 'Cn1c(=O)c2c(ncn2C)n(C)c1...
4,CHEMBL16687,{'canonical_smiles': 'Nc1nc2ccc(Cl)cc2c2nc(-c3...


## Preprocess and filter compound data

1. check for missing rows

In [48]:
compounds_df.isna().sum()

molecule_chembl_id     0
molecule_structures    0
dtype: int64

2. Get molecules with canonical SMILES

In [49]:
compounds_df.iloc[0].molecule_structures.keys()

dict_keys(['canonical_smiles', 'molfile', 'standard_inchi', 'standard_inchi_key'])

In [50]:
canonical_smiles = []

for i, compounds in compounds_df.iterrows():
    try:
        canonical_smiles.append(compounds["molecule_structures"]["canonical_smiles"])
    except KeyError:
        canonical_smiles.append(None)

compounds_df["smiles"] = canonical_smiles
compounds_df.drop("molecule_structures", axis=1, inplace=True)
compounds_df.head()

,molecule_chembl_id,smiles
0,CHEMBL428496,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...
1,CHEMBL273832,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...
2,CHEMBL535,CCN(CC)CCNC(=O)c1c(C)[nH]c(/C=C2\C(=O)Nc3ccc(F...
3,CHEMBL113,Cn1c(=O)c2c(ncn2C)n(C)c1=O
4,CHEMBL16687,Nc1nc2ccc(Cl)cc2c2nc(-c3ccco3)nn12


Summary of compound and bioactivity data

In [53]:
print(f"Bioactivities: {bioactivities_df.shape[0]}")
bioactivities_df.columns

Bioactivities: 4790


Index(['activity_id', 'assay_chembl_id', 'assay_description', 'assay_type',
       'molecule_chembl_id', 'relation', 'units', 'IC50', 'target_chembl_id',
       'target_organism', 'type'],
      dtype='object')

In [52]:
print(f"Compounds: {compounds_df.shape[0]}")
compounds_df.columns

Compounds: 4790


Index(['molecule_chembl_id', 'smiles'], dtype='object')

## Merge both datasets

In [55]:
# Merge DataFrames
output_df = pd.merge(
    bioactivities_df[["molecule_chembl_id", "IC50", "units"]],
    compounds_df,
    on="molecule_chembl_id",
)

# Reset row indices
output_df.reset_index(drop=True, inplace=True)

output_df.head(10)

,molecule_chembl_id,IC50,units,smiles
0,CHEMBL435507,2400.0,nM,CC1CN(c2cc(=O)c3ccc4ccccc4c3o2)CCO1
1,CHEMBL98350,2300.0,nM,O=c1cc(N2CCOCC2)oc2c(-c3ccccc3)cccc12
2,CHEMBL104468,13000.0,nM,O=c1cc(N2CCOCC2)oc2c1ccc1ccccc12
3,CHEMBL188678,5000.0,nM,O=c1cc(N2CCOCC2)oc2c(-c3cccc4c3sc3ccccc34)cccc12
4,CHEMBL379156,940.0,nM,O=C1NC(=O)/C(=C/c2ccc(-c3ccc(F)cc3O)o2)S1
5,CHEMBL428496,1.0,nM,COC[C@H]1OC(=O)c2coc3c2[C@@]1(C)C1=C(C3=O)[C@@...
6,CHEMBL437688,600.0,nM,O=C1NC(=S)S/C1=C\c1ccc(-c2cc([N+](=O)[O-])ccc2...
7,CHEMBL209159,340.0,nM,O=C1NC(=O)/C(=C/c2ccc(-c3cc(F)cc(F)c3O)s2)S1
8,CHEMBL211068,1000.0,nM,O=C1NC(=O)/C(=C/c2ccc(-c3ccc(O)cc3F)o2)S1
9,CHEMBL208696,1350.0,nM,O=C1NC(=O)/C(=C/c2ccc(-c3cc(C(=O)O)ccc3O)o2)S1


# Feature Engineering

## Add pIC50 Values for Enhanced Visualization and Processing

In this step, we augment our dataset by computing the pIC50 values for each compound. The pIC50 value is a logarithmic transformation of the IC50 value, which is commonly used in pharmacology to represent the potency of a compound as an inhibitor. By converting IC50 values to their corresponding pIC50 values, we enhance the interpretability and comparability of our data, particularly for compounds with varying potencies.

In [65]:
def convert_ic50_to_pic50(ic50_value):
    """
    Convert IC50 value to pIC50 value.
    """
    pic50_value = -math.log10(ic50_value * 10**-9) 
    return pic50_value

In [66]:
# Apply conversion to each row of the compounds DataFrame
output_df["pIC50"] = output_df.apply(lambda x: convert_ic50_to_pic50(x.IC50), axis=1)

In [67]:
output_df.head()

,molecule_chembl_id,IC50,units,smiles,pIC50,ROMol
0,CHEMBL1236962,0.040,nM,COc1ncc(-c2ccc3nccc(-c4ccnnc4)c3c2)cc1NS(=O)(=...,10.397940,<rdkit.Chem.rdchem.Mol object at 0x2af52ad53c10>
1,CHEMBL3577914,0.070,nM,COc1ncc(-c2ccc3nc(N)nc(-c4ccncc4)c3c2)cc1NS(=O...,10.154902,<rdkit.Chem.rdchem.Mol object at 0x2af52ad4e430>
2,CHEMBL3577911,0.095,nM,COc1ncc(-c2ccc3ncnc(-c4ccncc4)c3c2)cc1NS(=O)(=...,10.022276,<rdkit.Chem.rdchem.Mol object at 0x2af52ad4e580>
3,CHEMBL3577908,0.100,nM,COc1ncc(-c2ccc3nccc(-c4ccncc4)c3c2)cc1NS(=O)(=...,10.000000,<rdkit.Chem.rdchem.Mol object at 0x2af52ad4e6d0>
4,CHEMBL4878958,0.130,nM,COc1ncc2cc1NS(=O)(=O)c1ccc(F)c(c1)C(=O)NCc1ccc...,9.886057,<rdkit.Chem.rdchem.Mol object at 0x2af52ad86190>


In [68]:
# Add molecule column
PandasTools.AddMoleculeColumnToFrame(output_df, smilesCol="smiles")

In [69]:
# Sort molecules by pIC50
output_df.sort_values(by="pIC50", ascending=False, inplace=True)

In [70]:
# Reset index
output_df.reset_index(drop=True, inplace=True)
output_df.head(10)

,molecule_chembl_id,IC50,units,smiles,pIC50,ROMol
0,CHEMBL1236962,0.040,nM,COc1ncc(-c2ccc3nccc(-c4ccnnc4)c3c2)cc1NS(=O)(=...,10.397940,<rdkit.Chem.rdchem.Mol object at 0x2af52ad8d5f0>
1,CHEMBL3577914,0.070,nM,COc1ncc(-c2ccc3nc(N)nc(-c4ccncc4)c3c2)cc1NS(=O...,10.154902,<rdkit.Chem.rdchem.Mol object at 0x2af52ad8deb0>
2,CHEMBL3577911,0.095,nM,COc1ncc(-c2ccc3ncnc(-c4ccncc4)c3c2)cc1NS(=O)(=...,10.022276,<rdkit.Chem.rdchem.Mol object at 0x2af52ada52e0>
3,CHEMBL3577908,0.100,nM,COc1ncc(-c2ccc3nccc(-c4ccncc4)c3c2)cc1NS(=O)(=...,10.000000,<rdkit.Chem.rdchem.Mol object at 0x2af52ada5350>
4,CHEMBL4878958,0.130,nM,COc1ncc2cc1NS(=O)(=O)c1ccc(F)c(c1)C(=O)NCc1ccc...,9.886057,<rdkit.Chem.rdchem.Mol object at 0x2af52ada53c0>
5,CHEMBL3660111,0.140,nM,CCc1ncnc(N2CCN(S(C)(=O)=O)CC2)c1C#Cc1cnc(C)c(N...,9.853872,<rdkit.Chem.rdchem.Mol object at 0x2af52ada5120>
6,CHEMBL3577927,0.150,nM,O=S(=O)(Nc1cc(-c2cc3c(-c4ccncc4)ncnc3s2)cnc1Cl...,9.823909,<rdkit.Chem.rdchem.Mol object at 0x2af52ada50b0>
7,CHEMBL3660114,0.200,nM,Cc1ncc(C#Cc2c(C)ncnc2N2CCOC[C@@H]2C)cc1NS(=O)(...,9.698970,<rdkit.Chem.rdchem.Mol object at 0x2af52ada5190>
8,CHEMBL1082618,0.200,nM,COc1ccc2[nH]c(-c3c(C)nn(C)c3C)c(/C=C3/Oc4ccc(N...,9.698970,<rdkit.Chem.rdchem.Mol object at 0x2af52ada5200>
9,CHEMBL1808977,0.200,nM,CN(C)CCN(C)C(=O)c1ccc(NC(=O)Nc2ccc(-c3nc(O[C@H...,9.698970,<rdkit.Chem.rdchem.Mol object at 0x2af52ada5430>


In [71]:
output_df.to_csv("data/PIK3CA_compounds.csv")
